# Træning af RL til at balancere Cartpole

Denne notebook er en del af [AI for Beginners Curriculum](http://aka.ms/ai-beginners). Den er inspireret af [den officielle PyTorch-tutorial](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) og [denne Cartpole PyTorch-implementering](https://github.com/yc930401/Actor-Critic-pytorch).

I dette eksempel vil vi bruge RL til at træne en model til at balancere en stang på en vogn, der kan bevæge sig til venstre og højre på en horisontal skala. Vi vil bruge [OpenAI Gym](https://www.gymlibrary.ml/) miljøet til at simulere stangen.

> **Note**: Du kan køre koden fra denne lektion lokalt (f.eks. fra Visual Studio Code), hvor simuleringen vil åbne i et nyt vindue. Når du kører koden online, kan det være nødvendigt at lave nogle justeringer i koden, som beskrevet [her](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

Vi starter med at sikre, at Gym er installeret:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Lad os nu oprette CartPole-miljøet og se, hvordan vi kan arbejde med det. Et miljø har følgende egenskaber:

* **Handlingsrum** er det sæt af mulige handlinger, vi kan udføre ved hvert trin i simuleringen  
* **Observationsrum** er det rum af observationer, som vi kan foretage  


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Lad os se, hvordan simuleringen fungerer. Den følgende løkke kører simuleringen, indtil `env.step` ikke returnerer afslutningsflaget `done`. Vi vil tilfældigt vælge handlinger ved hjælp af `env.action_space.sample()`, hvilket betyder, at eksperimentet sandsynligvis vil fejle meget hurtigt (CartPole-miljøet afsluttes, når hastigheden af CartPole, dens position eller vinkel er uden for visse grænser).

> Simuleringen åbnes i et nyt vindue. Du kan køre koden flere gange og se, hvordan den opfører sig.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Du kan bemærke, at observationerne indeholder 4 tal. Disse er:
- Vognens position
- Vognens hastighed
- Stangens vinkel
- Stangens rotationshastighed

`rew` er den belønning, vi modtager ved hvert trin. Du kan se, at i CartPole-miljøet får du 1 point for hvert simuleringsskridt, og målet er at maksimere den samlede belønning, dvs. den tid, CartPole kan balancere uden at falde.

Under forstærkningslæring er vores mål at træne en **politik** $\pi$, der for hver tilstand $s$ fortæller os, hvilken handling $a$ vi skal udføre, så grundlæggende $a = \pi(s)$.

Hvis du ønsker en probabilistisk løsning, kan du tænke på politikken som noget, der returnerer et sæt sandsynligheder for hver handling, dvs. $\pi(a|s)$ ville betyde sandsynligheden for, at vi skal udføre handling $a$ i tilstand $s$.

## Policy Gradient-metoden

I den simpleste RL-algoritme, kaldet **Policy Gradient**, vil vi træne et neuralt netværk til at forudsige den næste handling.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Vi vil træne netværket ved at udføre mange eksperimenter og opdatere vores netværk efter hver kørsel. Lad os definere en funktion, der vil udføre eksperimentet og returnere resultaterne (den såkaldte **trace**) - alle tilstande, handlinger (og deres anbefalede sandsynligheder) og belønninger:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Du kan køre en episode med et utrænet netværk og observere, at den samlede belønning (AKA længden af episoden) er meget lav:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

En af de vanskelige aspekter ved policy gradient-algoritmen er at bruge **diskonterede belønninger**. Ideen er, at vi beregner vektoren af samlede belønninger ved hvert trin i spillet, og under denne proces diskonterer vi de tidlige belønninger ved hjælp af en koefficient $gamma$. Vi normaliserer også den resulterende vektor, fordi vi vil bruge den som vægt til at påvirke vores træning:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Nu skal vi i gang med den egentlige træning! Vi vil køre 300 episoder, og i hver episode vil vi gøre følgende:

1. Kør eksperimentet og indsamle sporingsdata.
2. Beregn forskellen (`gradients`) mellem de handlinger, der blev udført, og de forudsagte sandsynligheder. Jo mindre forskellen er, desto mere sikre er vi på, at vi har taget den rigtige handling.
3. Beregn diskonterede belønninger og multiplicér gradients med de diskonterede belønninger - det sikrer, at trin med højere belønninger har større indflydelse på det endelige resultat end trin med lavere belønninger.
4. Forventede målhandlinger for vores neurale netværk vil delvist blive taget fra de forudsagte sandsynligheder under kørslen og delvist fra de beregnede gradients. Vi vil bruge parameteren `alpha` til at bestemme, i hvilket omfang gradients og belønninger tages i betragtning - dette kaldes *læringsraten* for forstærkningsalgoritmen.
5. Til sidst træner vi vores netværk på tilstande og forventede handlinger og gentager processen.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Lad os nu køre episoden med rendering for at se resultatet:


In [ ]:
_ = run_episode(render=True)

Forhåbentlig kan du se, at stangen nu kan balancere ret godt!

## Actor-Critic Model

Actor-Critic-modellen er en videreudvikling af policy gradients, hvor vi bygger et neuralt netværk til at lære både politikken og de estimerede belønninger. Netværket vil have to output (eller du kan se det som to separate netværk):
* **Actor** vil anbefale den handling, der skal udføres, ved at give os sandsynlighedsfordelingen for tilstanden, som i policy gradient-modellen.
* **Critic** vil estimere, hvad belønningen ville være fra disse handlinger. Den returnerer de samlede estimerede belønninger i fremtiden i den givne tilstand.

Lad os definere en sådan model:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Vi ville være nødt til at ændre vores `discounted_rewards` og `run_episode` funktioner en smule:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Nu vil vi køre den primære træningssløjfe. Vi vil bruge en manuel netværkstræningsproces ved at beregne passende tabfunktioner og opdatere netværksparametre:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Vigtig pointe

Vi har set to RL-algoritmer i denne demo: simpel policy gradient og den mere sofistikerede actor-critic. Du kan se, at disse algoritmer arbejder med abstrakte begreber som tilstand, handling og belønning - hvilket gør dem anvendelige i meget forskellige miljøer.

Reinforcement learning giver os mulighed for at lære den bedste strategi til at løse problemet blot ved at kigge på den endelige belønning. Det faktum, at vi ikke behøver mærkede datasæt, gør det muligt for os at gentage simuleringer mange gange for at optimere vores modeller. Der er dog stadig mange udfordringer inden for RL, som du kan lære mere om, hvis du beslutter dig for at fokusere mere på dette interessante område inden for AI.



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi påtager os ikke ansvar for eventuelle misforståelser eller fejltolkninger, der opstår som følge af brugen af denne oversættelse.
